In [1]:
import os
from pathlib import Path

import polars as pl

In [2]:
dfs = []
folder = Path(os.getcwd()).parent / 'data/merged_dataset/synthetic'
files = os.listdir(folder)
print(files)
for f in files:
    if f.endswith('.tsv') and not f.startswith('_'):
        dfs.append(pl.read_csv(folder / f, separator='\t', quote_char='\"', infer_schema=False))

['.DS_Store', '_test_failed.tsv', 'medium_old', 'test.tsv', 'test_pc.tsv', 'test-alt.tsv', 'test-m3.tsv']


In [3]:
dfs

[shape: (301_645, 5)
 ┌─────────────────────┬─────────────────────┬─────────────────────┬──────────────┬─────────────────┐
 │ input               ┆ output              ┆ instruct            ┆ dataset_type ┆ dataloader_name │
 │ ---                 ┆ ---                 ┆ ---                 ┆ ---          ┆ ---             │
 │ str                 ┆ str                 ┆ str                 ┆ str          ┆ str             │
 ╞═════════════════════╪═════════════════════╪═════════════════════╪══════════════╪═════════════════╡
 │ input               ┆ output              ┆ instruct            ┆ dataset_type ┆ dataloader_name │
 │ Йдеться про         ┆ <p t="VERB">Йдеться ┆ Використай наведені ┆ 8            ┆ PapersDataset   │
 │ традицію феноменол… ┆ </p> <p t="…        ┆ нижче інст…         ┆              ┆                 │
 │ 3 З огляду на       ┆ <p t="NUM">3</p> <p ┆ Використай наведені ┆ 8            ┆ PapersDataset   │
 │ багатозначність …   ┆ t="ADP">З<…         ┆ нижче інст…   

In [4]:
merged = pl.concat(dfs)
shuffled_df = merged.sample(fraction=1, shuffle=True)

In [5]:
import math

train_df = shuffled_df.head(math.ceil(len(shuffled_df) * 0.60))
not_train_df = shuffled_df.tail(math.ceil(len(shuffled_df) * 0.40))
valid_df = not_train_df.head(math.ceil(len(not_train_df) * 0.40))
test_df = not_train_df.tail(math.ceil(len(not_train_df) * 0.60))


In [6]:
len(test_df), len(valid_df), len(not_train_df), len(train_df)

(383235, 255490, 638724, 958086)

In [8]:
large_folder = folder / 'large'
train_df.write_csv(large_folder / 'train.csv', separator='\t', quote_char='\"')
valid_df.write_csv(large_folder / 'valid.csv', separator='\t', quote_char='\"')
test_df.write_csv(large_folder / 'test.csv', separator='\t', quote_char='\"')

In [9]:
medium_folder = folder / 'medium'
medium_coef = 0.5
train_df.sample(fraction=medium_coef, shuffle=True).write_csv(medium_folder / 'train.csv', separator='\t', quote_char='\"')
valid_df.sample(fraction=medium_coef, shuffle=True).write_csv(medium_folder / 'valid.csv', separator='\t', quote_char='\"')
test_df.sample(fraction=medium_coef, shuffle=True).write_csv(medium_folder / 'test.csv', separator='\t', quote_char='\"')


In [10]:
small_folder = folder / 'small'
small_coef = 0.1
train_df.sample(fraction=small_coef, shuffle=True).write_csv(small_folder / 'train.csv', separator='\t', quote_char='\"')
valid_df.sample(fraction=small_coef, shuffle=True).write_csv(small_folder / 'valid.csv', separator='\t', quote_char='\"')
test_df.sample(fraction=small_coef, shuffle=True).write_csv(small_folder / 'test.csv', separator='\t', quote_char='\"')